# Metaprogramming in Julia

In [1]:
# abstract syntax tree (AST)
ex1 = parse("2+a*b-c")
dump(ex1)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 2
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Symbol a
            3: Symbol b
          typ: Any
      typ: Any
    3: Symbol c
  typ: Any


## Computing harmonic mean in macro

In [2]:
macro mean(arr...)
    ex = Expr(:call, :/, 1, arr[1])
    for i = 2:length(arr)
         ex = Expr(:call, :+, ex, Expr(:call, :/, 1, arr[i]))
    end
    
    Expr(:call, :/, length(arr), ex)
end

@mean (macro with 1 method)

In [3]:
@mean(1,2,3)

1.6363636363636365

In [4]:
macroexpand(:(@mean(2,2,5,7)))

:(4 / (((1 / 2 + 1 / 2) + 1 / 5) + 1 / 7))

## Automatic differentiation

In [2]:
using Match

function mult(arr)
    num = 0
    mul = 1
    
    for elem in arr
        @match elem begin
            :x => num = num + 1
            n => mul = mul * n
        end
    end
    
    return :($(mul*num) * :x ^ $(num-1))
end

function autodiff(ex:: Symbol)
    @match ex begin
        :x => return 1
        n => return 0
    end
end

function autodiff(ex:: Expr)
    
    res = Expr(:call, :+, 0, 0)
    
    @match ex.args begin
        [:+, n...] =>  for i = 1:length(n)
            ex = autodiff(n[i])
            res = Expr(:call, :+, ex, res)
          end
        [:-, n...] => for i = 1:length(n)
            ex = autodiff(n[i])
            res = Expr(:call, :-, ex, res)
          end
        [:*, n...] => res = Expr(:call, :+, mult(n), res)
        [:/, n...] => res = Expr(:call, :/, 1, n[2])
        [:x] => 1
    end
    
    return res
end

function autodiff(ex:: Int64)
     return 0
end

autodiff (generic function with 3 methods)

In [7]:
autodiff(:(x + x*x*x))

:((3 * :x ^ 2 + (0 + 0)) + (1 + (0 + 0)))

In [8]:
autodiff(:(4 * x))

:(4 * :x ^ 0 + (0 + 0))

In [9]:
autodiff(:(3 * x * x * x + 2 * x * x - 5 * x - 10))

:(0 - (((5 * :x ^ 0 + (0 + 0)) - (((4 * :x ^ 1 + (0 + 0)) + ((9 * :x ^ 2 + (0 + 0)) + (0 + 0))) - (0 + 0))) - (0 + 0)))

In [4]:
autodiff(:(x / 2 + 3 * x))

:((3 * :x ^ 0 + (0 + 0)) + (1 / 2 + (0 + 0)))